# This is NOT the main Script
## This notebook was used to help build the main script.
### Selenium Webscraping of Fidelity Stock Research

More information on how the notebook works will be added later.

In [1]:
# imports
from selenium.webdriver import Firefox
import time
from selenium.webdriver.common.by import By
import pandas as pd
import re
import yfinance as yf
import extract

In [ ]:
# coming later; this cell, and likely the ones underneath can be used to read the csv data, create a new excel document to write to, and copy over the relevant data into the new excel document.

In [2]:
# open the browser, navigate to Fidelity.com
# this cell assumes that you have added the webdriver to your system path
driver = Firefox()
driver.get("https://www.fidelity.com/")

After the browser navigates to the Fidelity website, go to the browser and log in as you normally would. Once logged in, you can run the next cell.

In [ ]:
# in the final version of this, this is approximately where we would start loading pages and extracting data. During development, this will be a series of that accomplish individual tasks, which will be compiled together into a single script later.

In [3]:
# this loads the stock research page for V
driver.get("https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/research?symbol=DHI&appCode=&optInNRE=")
# a lot of the elements on the Fidelity research pages won't load unless they are scrolled past in the browser. This next line automates that step. Thanks to pythonbasics.org for this trick.
time.sleep(10)
driver.execute_script("window.scrollTo(0, 0)")
time.sleep(1)
driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
time.sleep(4)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [8]:
# first, determine if the security is a stock, ETF, mutual fund, or other
# TO FIX: this does not work on Mutual Fund pages
try:
    sec_type = driver.find_element_by_class_name('breadcrumb-div')
except:
    try:
        print('must be a MF')
        sec_type = driver.find_element_by_xpath('/html/body/div[3]/div/div/div/header-template/div[1]/div[1]')
    except:
        print("Must not be a MF either, can't find anything")
try:
    sec_type = sec_type.text
    print(sec_type)
except AttributeError:
    pass
sec_type.strip("»")
sec_type = sec_type.split()
for x in sec_type:
    if x == 'Stocks':
        sec_type = x
        break
    elif x == 'ETFs':
        sec_type = x
        break
    elif x == 'Mutual':
        sec_type = 'Mutual Funds'
        break
sec_type

Research > Stocks >
Video Help/Glossary


'Stocks'

# analysis items for stocks
analysis_items = driver.find_element_by_class_name('analysis-items')
analysis_items.text

In [8]:
# regex to extract useful data
valuation_pos = analysis_items.text.index('Valuation\ncurrent =')
valuation = analysis_items.text[(valuation_pos + 19) : (valuation_pos + 23)]
valuation = re.findall(r'\d+', valuation)
int(valuation[0])

75

In [9]:
# regex for Quality
quality_pos = analysis_items.text.index("Quality")
quality = analysis_items.text[(quality_pos + 18) : (quality_pos + 22)]
quality = re.findall(r'\d+', quality)
int(quality[0])

90

In [10]:
# regex for Growth Stability
growth_stability_pos = analysis_items.text.index("Growth")
growth_stability = analysis_items.text[(growth_stability_pos + 23) : (growth_stability_pos + 30)]
growth_stability = re.findall(r"\d+", growth_stability)
int(growth_stability[0])

88

In [11]:
# regex for Financial Health
fin_health_pos = analysis_items.text.index("Health")
fin_health = analysis_items.text[(fin_health_pos + 15): (fin_health_pos + 20)]
fin_health = re.findall(r"\d+", fin_health)
int(fin_health[0])

75

In [12]:
# equity summary score
equity_summ_score = driver.find_element_by_class_name('rating-score')
equity_summ_score.text

'7.7'

In [13]:
# 52 Week Performance
fifty_two_week = driver.find_elements_by_id('box14')
ftw_perf = fifty_two_week[0].text.splitlines()
float(ftw_perf[1].strip('+').strip('%'))

77.33

In [14]:
# Div yield
dividends = driver.find_elements_by_id('box11')
dividends = dividends[0].text.split()
for item in dividends:
    if '$' in item:
        div_amt = float(item.strip('$'))
    if '%' in item:
        div_yield = float(item.strip('%'))
div_amt, div_yield

(0.82, 0.68)

In [15]:
# recognia short term outlook
# unsure how well this will work w/ other things
# the xpath may be a problematic way to do this
short_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[1]/div[2]')
short_list = short_term[0].get_attribute('innerHTML').split()
for i in range(0, len(short_list)):
    if short_list[i] == 'bold':
        recognia_short_term = short_list[i + 1]
        break
recognia_short_term = re.findall(r'[a-z]*', recognia_short_term)
recognia_short_term[0]

'weak'

In [16]:
# attempting to apply same method to mid-term
# i should convert this to a function perhaps
mid_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[2]')
mid_list = mid_term[0].get_attribute('innerHTML').split()
for i in range(0, len(mid_list)):
    if mid_list[i] == 'bold':
        recognia_mid_term = mid_list[i + 1]
        break
recognia_mid_term = re.findall(r'[a-z]*', recognia_mid_term)
recognia_mid_term[0]

'weak'

In [17]:
# attempting to apply same method to mid-term
# i should convert this to a function perhaps
long_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[3]')
long_list = long_term[0].get_attribute('innerHTML').split()
for i in range(0, len(long_list)):
    if long_list[i] == 'bold':
        recognia_long_term = long_list[i + 1]
        break
recognia_long_term = re.findall(r'[a-z]*', recognia_long_term)
recognia_long_term[0]

'neutral'

In [52]:
# this is an attempt to condense the three separate code blocks down to one, in an attempt to not rewrite the same code twice
recognia_fixed_dict = {
    'Recognia Short Term': '/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[1]/div[2]',
    'Recognia Intermediate Term': '/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[2]',
    'Recognia Long Term': '/html/body/div[3]/div[2]/div[1]/div[11]/div/div/div/div/section/div/div[3]'
}

recognia_dict = dict()

for key, val in recognia_fixed_dict.items():
    element = driver.find_elements_by_xpath(val)
    elem_list = element[0].get_attribute('innerHTML').split()
    for i in range(0, len(elem_list)):
        if elem_list[i] == 'bold':
            result = elem_list[i + 1]
            result = re.findall(r'[a-z]*', result)
            recognia_dict[key] = result[0]
            break
recognia_dict

# note on 3/2/2021 08:28: to make this work w/ ETFs, I'll just have an ETF part of the if statement and create a different dict for the ETF xpaths

{'Recognia Short Term': 'weak',
 'Recognia Intermediate Term': 'strong',
 'Recognia Long Term': 'strong'}

In [3]:
# next, now that we have all of the information we want from the first page, we move to the key stats page
driver.get('https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/keyStatistics?symbol=AAPL')

# then, we scroll thru the page like we did earlier to ensure all elements have rendered to the page and are thus accessible for us to extract

time.sleep(10)
driver.execute_script("window.scrollTo(0, 0)")
time.sleep(1)
driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
time.sleep(4)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [ ]:
# NOTE: this will be deprecated; see the cell below, which works in a much more efficient way

# now, extract the necessary data; start w/ P/E ratio TTM
# the first number in each row describes the actual company statistic, the second is the industry average
key_stats = driver.find_elements_by_class_name('datatable-component')
key_stats = key_stats[0].text.splitlines()
# key_stats
for stat in key_stats:
    if 'P/E (Trailing Twelve Months)' in stat:  # P/E TTM
        pe_ttm = stat.split()
        for x in pe_ttm:
            if '.' in x:
                pe_ttm = float(x)   # we only want the first value that would meet this criteria; this would be the actual company's P/E, not the industry one
                break
    elif 'P/E (5-Year Average)' in stat:    # P/E 5 year average
        pe_five_yr = stat.split()
        for x in pe_five_yr:
            if '.' in x:
                pe_five_yr = float(x)
                break
    elif 'Price/Cash Flow (Most' in stat:   #P/CF most recent quarter
        pcf_quarter = stat.split()
        for x in pcf_quarter:
            if '.' in x:
                pcf_quarter = float(x)
                break
    elif 'Price/Cash Flow (TTM)' in stat:
        pcf_ttm = stat.split()
        for x in pcf_ttm:
            if '.' in x:
                pcf_ttm = float(x)
                break
pe_ttm, pe_five_yr, pcf_quarter, pcf_ttm


In [19]:
# the above cell works, but I think there is likely a more efficient way to write it using dicts
stock_dict = dict()

# normally, the values would be added stock by stock, but since we're mainly developing using V, I will just manually add it
stock_dict['V'] = dict()

# now, we need a list of statistics that we are looking for to scrape. We're going to use a tuple, because we won't be changing it.
stats = ('P/E (Trailing Twelve Months)', 'P/E (5-Year Average)', 'Price/Cash Flow (Most Recent Quarter)', 'Price/Cash Flow (TTM)', 'Price/Sales (Most Recent Quarter)', 'Price/Sales (TTM)', 'Price/Book')

# extract data from the page
key_stats = driver.find_elements_by_class_name('datatable-component')
key_stats = key_stats[0].text.splitlines()

# iterate thru data and add to dict
# we would normally do this via iteration of all symbols
for key, val in stock_dict.items():
    for stat in key_stats:
        for x in stats:
            if x in stat:
                y = stat.split()
                for z in y:
                    if '.' in z:
                        val[x] = float(z)
                        break

# this next part is to test to see if the above code worked
for key,val in stock_dict.items():
    for x, y in val.items():
        print(x,y)

P/E (Trailing Twelve Months) 32.84
P/E (5-Year Average) 18.82
Price/Cash Flow (Most Recent Quarter) 16.23
Price/Cash Flow (TTM) 27.64
Price/Sales (Most Recent Quarter) 4.58
Price/Sales (TTM) 7.03
Price/Book 29.31


In [13]:
# extract overall price to debt, stocks only:
key_stats = driver.find_elements_by_class_name('datatable-component')
key_stats = key_stats[4].text.splitlines()
# key_stats

debt_stats = ('Total Debt/Equity (Most Recent Quarter, Annualized)', 'Total Debt/Equity (TTM)')
debt_dict = dict()

for stat in debt_stats:
    for x in key_stats:
        if stat in x:
            x = x.split()
            for text in x:
                if '%' in text:
                    debt_dict[stat] = float(text.strip('%'))
                    break
debt_dict

{'Total Debt/Equity (Most Recent Quarter, Annualized)': 169.19,
 'Total Debt/Equity (TTM)': 168.37}

In [ ]:
# conver the dict to a DF
# I think this may be the easiest/fastest way to do this given my data and my extraction methods

stock_df = pd.DataFrame.from_dict(stock_dict, orient='index')
stock_df

In [42]:
# finding price targets
# yfinance does not provide price targets, so I'll need to scrape that data myself

# first, need to open a new tab, then change the focus to that new tab
driver.execute_script('''window.open("https://finance.yahoo.com/quote/V?p=V&.tsrc=fin-srch", "_blank");''')
time.sleep(3)
driver.switch_to_window(driver.window_handles[1])

In [45]:
# now extract that data using the following xpath
price_target = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/div/div[2]/div[2]/table/tbody/tr[8]/td[2]/span')
float(price_target[0].text)

IndexError: list index out of range

In [44]:
# then, switch back to fidelity
other_window = driver.window_handles[0] # switches back to the first open window (fidelity)
# other_window
driver.switch_to.window(window_name=other_window)

## ETF Data Extraction Below

In [20]:
# this is for retrieving data for ETFs
# this loads the stock research page for V
driver.get("https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/research?symbol=TAN&appCode=&optInNRE=")
# a lot of the elements on the Fidelity research pages won't load unless they are scrolled past in the browser. This next line automates that step. Thanks to pythonbasics.org for this trick.
time.sleep(10)
driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
time.sleep(4)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [26]:
# first, get some of the key stats from this; a lot of this is different from stocks, so ultimately, we'll have a bit less data to work w/
etf_stats = ('Price / Earnings (TTM)', 'Price / Book', 'Price / Sales', 'Price / Cash Flow', 'Distribution Yield')

key_stats = driver.find_elements_by_id('keyStatistics-equity-data-table')
key_stats = key_stats[0].text.splitlines()

etf_stats_dict = dict()
for x in range(len(key_stats)):
    for y in etf_stats:
        if y in key_stats[x]:
            if 'Distribution Yield' in key_stats[x]:
                etf_stats_dict['etf_distribution_yield'] = float(key_stats[x + 2].strip('%'))
            else:
                etf_stats_dict[y] = float(key_stats[x + 1])

etf_stats_dict
            


{'Price / Earnings (TTM)': 47.24,
 'Price / Book': 4.34,
 'Price / Sales': 3.5,
 'Price / Cash Flow': 14.82,
 'etf_distribution_yield': 0.08}

In [31]:
ftwk_perf_etf = driver.find_elements_by_id('box14etf')
ftwk_perf_etf = ftwk_perf_etf[0].text.splitlines()
ftwk_perf_etf = ftwk_perf_etf[1].strip('+').strip('%')
ftwk_perf_etf

'177.67'

In [35]:
# recognia for ETFs
# short term

recognia_short_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[7]/div/div/div/div/section/div/div[1]')
short_list = recognia_short_term[0].get_attribute('innerHTML').split()
for i in range(0, len(short_list)):
    if short_list[i] == 'bold':
        recognia_short_term = short_list[i + 1]
        break
recognia_short_term = re.findall(r'[a-z]*', recognia_short_term)
recognia_short_term[0]

'weak'

In [36]:
# recognia for ETFs
# mid term

recognia_mid_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[7]/div/div/div/div/section/div/div[2]')
mid_list = recognia_mid_term[0].get_attribute('innerHTML').split()
for i in range(0, len(mid_list)):
    if mid_list[i] == 'bold':
        recognia_mid_term = mid_list[i + 1]
        break
recognia_mid_term = re.findall(r'[a-z]*', recognia_mid_term)
recognia_mid_term[0]

'weak'

In [37]:
# recognia for ETFs
# long term

recognia_long_term = driver.find_elements_by_xpath('/html/body/div[3]/div[2]/div[1]/div[7]/div/div/div/div/section/div/div[3]')
long_list = recognia_long_term[0].get_attribute('innerHTML').split()
for i in range(0, len(long_list)):
    if long_list[i] == 'bold':
        recognia_long_term = long_list[i + 1]
        break
recognia_long_term = re.findall(r'[a-z]*', recognia_long_term)
recognia_long_term[0]

'strong'

## Mutual Fund Data Extraction

In [39]:
# fetch using the same link, b/c apparently that works?
driver.get("https://snapshot.fidelity.com/fidresearch/snapshot/landing.jhtml#/research?symbol=FFFHX&appCode=&optInNRE=")
# a lot of the elements on the Fidelity research pages won't load unless they are scrolled past in the browser. This next line automates that step. Thanks to pythonbasics.org for this trick.
time.sleep(10)
driver.execute_script("window.scrollTo(0,(document.body.scrollHeight / 2))")
time.sleep(4)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [43]:
# really the only metric that we can extract that will fit into our excel document is 1-yr performance
mf_ftwk_perf = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div/div[1]/div[2]/performance-template/div/div[2]/div/div/div[2]/div[1]/p[2]')
mf_ftwk_perf = float(mf_ftwk_perf[0].text.strip('+').strip('%'))
mf_ftwk_perf

32.94

## Testing Functions in the extract.py file.

In [4]:
sec_type = extract.determine_security_type(driver)
print(sec_type)
# altho this output looks weird it works?

Stocks


In [10]:
extract.stock_div_yield(driver)

(0.8, 1.02)

In [11]:
extract.stock_equity_sum_score(driver)

6.8

In [20]:
sec_type
sec_type == sec_type

True

In [5]:
extract.recognia_analysis(driver, sec_type)

{'Recognia Short Term': 'weak',
 'Recognia Intermediate Term': 'strong',
 'Recognia Long Term': 'strong'}

In [6]:
extract.stock_valuation(driver)

63

In [7]:
extract.stock_quality(driver)

37

In [8]:
extract.stock_growth_stability(driver)

77

In [9]:
extract.stock_financial_health(driver)

96

In [12]:
extract.stock_key_stats(driver, 'DHI')
# needs tested again when the kernel is re-init

({'10.65': 10.65,
  '11.97': 11.97,
  '8.85': 8.85,
  '10.22': 10.22,
  '1.21': 1.21,
  '1.29': 1.29,
  '2.50': 2.5},
 {'Total Debt/Equity (Most Recent Quarter, Annualized)': 34.13,
  'Total Debt/Equity (TTM)': 37.67})

In [13]:
# prep for testing yf stuff
driver.execute_script('''window.open("https://finance.yahoo.com/quote/V?p=V&.tsrc=fin-srch", "_blank");''')
time.sleep(3)

In [15]:
extract.stock_one_yr_price_target(driver, 'DHI')
# needs new test, possibly fixed

NoSuchElementException: Message: Web element reference not seen before: {"element-6066-11e4-a52e-4f735466cecf":"02799716-a0f5-4c57-8484-3b1b9f363e03"}


## Fixing Things, before re-init kernel to test above

In [25]:
driver.switch_to_window(driver.window_handles[1])
driver.get(f'https://finance.yahoo.com/quote/AAPL?p=AAPL&.tsrc=fin-srch')
time.sleep(5)   # I don't think this will need automated scrolling down the page, not sure though.
price_target = driver.find_elements_by_id('quote-summary')
price_target = price_target[0].text.splitlines()
price_target
for stat in price_target:
    if '1y Target' in stat:
        stat = stat.split()
        for x in stat:
            if '.' in x:
                price_target = float(x)
                break
price_target
# time.sleep(5)
# other_window = driver.window_handles[0] # switches back to the first open window # (fidelity)
# # other_window
# driver.switch_to.window(window_name=other_window)
# float(price_target[0].text)

151.75